# Phase 3: Llama-3-8B QLoRA 파인튜닝

`refine_training_dataset.ipynb`에서 생성한 정제된 `training_dataset_refined.jsonl`을 사용하여 Unsloth 기반 QLoRA 파인튜닝을 수행하고, 16bit 병합 모델을 Google Drive에 저장합니다.

**개선 사항**:
- Loss Masking 적용 (Response 부분만 학습)
- Epoch 2로 조정 (과적합 방지)
- LoRA rank 32로 증가 (표현력 향상)


## ⚙️ 사전 조건
- `refine_training_dataset.ipynb` 실행으로 `outputs/training_dataset_refined.jsonl` 존재
- Colab T4 이상 GPU 할당
- 런타임 초기화 시 아래 설치 셀 재실행


---
## 0. 필수 패키지 설치 (Unsloth + TRL)


In [1]:
import torch
import time

major_version, minor_version = torch.cuda.get_device_capability()
print(f"CUDA Capability: {major_version}.{minor_version}")

start_time = time.time()

# 1. Unsloth 설치
print("\n[1/4] Unsloth 설치 중...")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --no-warn-script-location
print("✅ Unsloth 설치 완료\n")

# 2. peft 업그레이드
print("[2/4] peft 업그레이드 중...")
!pip install --upgrade --no-deps "peft>=0.12.0" --no-warn-script-location
print("✅ peft 업그레이드 완료\n")

# 3. trl 다운그레이드 (0.8.6으로 - DataCollatorForCompletionOnlyLM 호환)
print("[3/4] trl 다운그레이드 중...")
!pip install --no-deps "trl==0.8.6" --no-warn-script-location
print("✅ trl 다운그레이드 완료\n")

# 4. bitsandbytes 설치
print("[4/4] bitsandbytes 설치 중...")
!pip install --no-deps "bitsandbytes" --no-warn-script-location
print("✅ bitsandbytes 설치 완료\n")

# 설치 확인
print("\n=== 설치 확인 ===")
try:
    import unsloth
    print(f"✅ unsloth: 설치됨")
except Exception as e:
    print(f"❌ unsloth: {e}")

try:
    import peft
    print(f"✅ peft: {peft.__version__}")
except Exception as e:
    print(f"❌ peft: {e}")

try:
    import accelerate
    print(f"✅ accelerate: {accelerate.__version__}")
except Exception as e:
    print(f"❌ accelerate: {e}")

try:
    import bitsandbytes
    print(f"✅ bitsandbytes: {bitsandbytes.__version__}")
except Exception as e:
    print(f"❌ bitsandbytes: {e}")

try:
    import trl
    print(f"✅ trl: {trl.__version__}")
    from trl import DataCollatorForCompletionOnlyLM
    print("✅ DataCollatorForCompletionOnlyLM import 성공")
except Exception as e:
    print(f"❌ trl: {e}")

elapsed_time = time.time() - start_time
print(f"\n총 설치 시간: {elapsed_time/60:.1f}분")
print("✅ 모든 패키지 설치 완료")

CUDA Capability: 8.9

[1/4] Unsloth 설치 중...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ud1gmekk/unsloth_fc3c0a7ed8a14d9989bf0699c2cec1b2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ud1gmekk/unsloth_fc3c0a7ed8a14d9989bf0699c2cec1b2
  Resolved https://github.com/unslothai/unsloth.git to commit d1e312dcdc57bf020aa0f6da810226efe79cd69a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Unsloth 설치 완료

[2/4] peft 업그레이드 중...
✅ peft 업그레이드 완료

[3/4] trl 다운그레이드 중...
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0
✅ trl 다운그레이드 완료

[4/4] bitsandbytes 설치 중...
✅ bitsandbytes 설치 완료


=== 설치 확인 ===
🦥 Unsloth: Will patch yo

In [2]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 1. 경로 및 데이터 로드


In [3]:
from pathlib import Path
import json
from datasets import Dataset

PROJECT_ROOT = Path("/content/drive/MyDrive//board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
MODEL_DIR = PROJECT_ROOT / "models_refined"
CHECKPOINT_DIR = PROJECT_ROOT / "checkpoints_refined"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# 정제된 데이터셋 사용
TRAIN_JSONL = OUTPUT_DIR / "training_dataset_refined.jsonl"
if not TRAIN_JSONL.exists():
    raise FileNotFoundError(f"{TRAIN_JSONL}가 없습니다. refine_training_dataset.ipynb를 먼저 실행하세요.")

training_data = []
with open(TRAIN_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            training_data.append(json.loads(line))

print(f"✅ 데이터셋 로드 완료: {len(training_data)}개 레코드")


✅ 데이터셋 로드 완료: 5752개 레코드


In [4]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    # Llama-3의 EOS 토큰 직접 지정
    eos_token = "<|eot_id|>"

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}{eos_token}"""
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_list(training_data)
dataset = dataset.map(formatting_prompts_func, batched=True)
# 데이터 셔플 추가 (과적합 방지)
dataset = dataset.shuffle(seed=42)
print("✅ 데이터셋 템플릿 적용 및 셔플 완료")


Map:   0%|          | 0/5752 [00:00<?, ? examples/s]

✅ 데이터셋 템플릿 적용 및 셔플 완료


---
## 2. Unsloth 모델 로드 및 QLoRA 준비


In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # 16 -> 32로 증가 (표현력 향상)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

print("✅ Unsloth 모델 로드 완료")


==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ Unsloth 모델 로드 완료


---
## 3. SFTTrainer 설정 및 학습


In [7]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
import torch

# 이제 정상적으로 Import 될 것입니다.
print("✅ Import 성공!")

# Loss Masking 적용
response_template = "### Response:\n"
collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer
)

# Trainer 설정 (기존 코드와 동일)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    data_collator=collator,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir=str(CHECKPOINT_DIR),
        save_strategy="epoch",
        save_total_limit=3,
    ),
)

trainer.train()

✅ Import 성공!


Map:   0%|          | 0/5752 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,752 | Num Epochs = 2 | Total steps = 1,438
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
1,3.402200
2,3.704900
3,3.120600
4,3.356800
5,3.191600
6,3.374100
7,3.481600
8,3.363700
9,2.849600
10,2.595500


TrainOutput(global_step=1438, training_loss=2.615666801558085, metrics={'train_runtime': 5797.1788, 'train_samples_per_second': 1.984, 'train_steps_per_second': 0.248, 'total_flos': 1.7513633513875046e+17, 'train_loss': 2.615666801558085, 'epoch': 2.0})

---
## 4. 모델 저장 (LoRA + 16bit 병합)


In [8]:
OUTPUT_MODEL_DIR = MODEL_DIR / "llama3_popular_post_lora"
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# LoRA 어댑터 저장
model.save_pretrained(str(OUTPUT_MODEL_DIR))
tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
print(f"✅ LoRA 어댑터 저장 완료: {OUTPUT_MODEL_DIR}")

# 16bit 병합 모델 저장
MERGED_DIR = OUTPUT_MODEL_DIR / "merged_16bit"
FastLanguageModel.for_inference(model)
model.save_pretrained_merged(
    MERGED_DIR,
    tokenizer,
    save_method="merged_16bit",
)
print(f"✅ 16bit 병합 모델 저장 완료: {MERGED_DIR}")


✅ LoRA 어댑터 저장 완료: /content/drive/MyDrive/board_crawling/models_refined/llama3_popular_post_lora


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [01:16<00:00, 19.06s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:34<00:00, 23.56s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/board_crawling/models_refined/llama3_popular_post_lora/merged_16bit`
✅ 16bit 병합 모델 저장 완료: /content/drive/MyDrive/board_crawling/models_refined/llama3_popular_post_lora/merged_16bit


> 💡 **Tip**: 필요 시 `MERGED_DIR`를 `.zip`으로 압축하여 Google Drive/Hub 등에 업로드하면 Colab 세션이 종료되어도 보관할 수 있습니다.


---
## 5. 다음 단계
- `phase4_generation.ipynb`에서 위 모델을 로드하여 추론/인기글 생성을 수행합니다.
- 추가로 Hugging Face Hub에 업로드하려면 `model.push_to_hub()`를 활용하세요.
